- Fetch Departures for a specific Stop

In [29]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime

# Load API key from environment variables
load_dotenv()
API_KEY = os.getenv("TRAFFICLABS_API_KEY")

# Fetch stops for a given location
def fetch_stops(city_name):
    """
    Fetch stops in the specified city or location using the ResRobot API.
    """
    base_url = "https://api.resrobot.se/v2.1/location.name"
    params = {
        "input": city_name,
        "format": "json",
        "accessId": API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        stop_data = []
        data = response.json()
        if "stopLocationOrCoordLocation" in data:
            for item in data["stopLocationOrCoordLocation"]:
                if "StopLocation" in item:
                    stop = item["StopLocation"]
                    stop_data.append({
                        "Name": stop.get("name", "Unknown"),
                        "Latitude": stop.get("lat"),
                        "Longitude": stop.get("lon"),
                        "ID": stop.get("id")
                    })
        return stop_data
    else:
        print(f"Failed to fetch stops. Status Code: {response.status_code}")
        return []

# Fetch departures for a specific stop to a destination
def fetch_departures(from_stop_id, to_stop_id):
    """
    Fetch departures for a given stop ID using the ResRobot API.
    """
    base_url = "https://api.resrobot.se/v2.1/departureBoard"
    params = {
        "id": from_stop_id,
        "format": "json",
        "accessId": API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        departures = response.json().get("Departure", [])
        filtered_departures = []
        
        # Filter departures that go to the destination stop
        for departure in departures:
            if "direction" in departure and departure["direction"] == to_stop_id:
                filtered_departures.append(departure)
        
        return filtered_departures
    else:
        print(f"Failed to fetch departures. Status Code: {response.status_code}")
        return []

# Process and format the departure data
def process_departures(departures):
    """
    Process departures to extract relevant information like transport type, line, origin, and departure time.
    """
    current_time = datetime.now()
    processed_data = []

    for departure in departures:
        product = departure.get("Product", {})
        if isinstance(product, list):  # Handle cases where Product is a list
            product = product[0] if product else {}

        transport_type = product.get("catOutL", "Unknown")
        line = product.get("line", "Unknown")
        origin = departure.get("direction", "Unknown")
        time_str = f"{departure['date']} {departure['time']}"
        departure_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
        minutes_left = (departure_time - current_time).total_seconds() // 60

        processed_data.append({
            "Transport Type": transport_type,
            "Line": line,
            "Origin": origin,
            "Departure Time": departure_time.strftime("%H:%M:%S"),
            "Minutes Left": max(0, int(minutes_left)),
        })

    return processed_data

# Display the departures in a readable format
def display_departures(departures):
    """
    Display departure information in a readable format.
    """
    if not departures:
        print("No departures available for the specified stop.")
        return

    print("\nDepartures:")
    for departure in departures:
        print(f"{departure['Transport Type']} {departure['Line']} to {departure['Origin']} departs at {departure['Departure Time']} ({departure['Minutes Left']} minutes left)")

# Main function to orchestrate the process
def main():
    # Step 1: Input the city name
    city_name = input("Enter the name of the city or location: ").strip()
    if not city_name:
        print("City name cannot be empty.")
        return

    # Step 2: Fetch stops in the city
    print(f"Fetching stops in {city_name}...")
    stops = fetch_stops(city_name)
    if not stops:
        print(f"No stops found for {city_name}. Exiting...")
        return

    # Step 3: Display stops and let the user select the starting stop
    print("\nAvailable Stops:")
    for index, stop in enumerate(stops):
        print(f"{index + 1}. {stop['Name']} (ID: {stop['ID']})")
    try:
        from_stop_choice = int(input("\nSelect the stop where you're currently at by entering the corresponding number: ")) - 1
        if from_stop_choice < 0 or from_stop_choice >= len(stops):
            print("Invalid choice. Exiting...")
            return
    except ValueError:
        print("Invalid input. Exiting...")
        return

    from_stop = stops[from_stop_choice]
    from_stop_id = from_stop["ID"]
    print(f"\nYou are at: {from_stop['Name']} (ID: {from_stop_id})")

    # Step 4: Let the user input their destination stop
    destination_name = input("\nEnter your destination stop name: ").strip()
    if not destination_name:
        print("Destination stop cannot be empty.")
        return

    # Step 5: Fetch stops in the city again to find destination stop
    print(f"Fetching stops for {destination_name}...")
    destination_stops = fetch_stops(city_name)
    destination_stop = None

    for stop in destination_stops:
        if destination_name.lower() in stop["Name"].lower():
            destination_stop = stop
            break

    if destination_stop is None:
        print("Destination stop not found. Exiting...")
        return

    destination_stop_id = destination_stop["ID"]
    print(f"Destination stop found: {destination_stop['Name']} (ID: {destination_stop_id})")

    # Step 6: Fetch and process departures from the selected stop to the destination
    departures = fetch_departures(from_stop_id, destination_stop_id)
    if departures:
        processed_departures = process_departures(departures)
        display_departures(processed_departures)
    else:
        print(f"No departures found from {from_stop['Name']} to {destination_stop['Name']}.")

if __name__ == "__main__":
    main()


Fetching stops in Partille...

Available Stops:
1. PARTILLE (ID: A=1@O=PARTILLE@X=12102861@Y=57740870@U=1@L=740098044@B=1@p=1737693407@)
2. Partille station (ID: A=1@O=Partille station@X=12102861@Y=57740870@U=1@L=740000132@B=1@p=1737693407@)
3. Partille arena (ID: A=1@O=Partille arena@X=12118016@Y=57741050@U=1@L=740072673@B=1@p=1737693407@)
4. Partille centrum (ID: A=1@O=Partille centrum@X=12105252@Y=57738623@U=1@L=740015667@B=1@p=1737693407@)
5. Partille Hålldammen (ID: A=1@O=Partille Hålldammen@X=12106429@Y=57733562@U=1@L=740015671@B=1@p=1737693407@)
6. Partille port (ID: A=1@O=Partille port@X=12122808@Y=57740978@U=1@L=740059976@B=1@p=1737693407@)
7. Persplatsen (Partille kn) (ID: A=1@O=Persplatsen (Partille kn)@X=12065492@Y=57720950@U=1@L=740060009@B=1@p=1737693407@)
8. Jonsered station (Partille kn) (ID: A=1@O=Jonsered station (Partille kn)@X=12173803@Y=57749958@U=1@L=740015675@B=1@p=1737693407@)
9. Jonsered Jons väg (Partille kn) (ID: A=1@O=Jonsered Jons väg (Partille kn)@X=121741